In [2]:
import ollama

model = "llama3.2:1b"
message_content = 'why do leaves change in the fall?'
message = {'role': 'user', 'content': message_content}

# Try to call ollama directly - this assumes Ollama is already installed, along with the target model
ollama.chat(model=model,messages=[message])

{'model': 'llama3.2:1b',
 'created_at': '2024-10-11T00:33:17.799145Z',
 'message': {'role': 'assistant',
  'content': "Leaves changing color in the fall is a natural process that occurs when the days get shorter and the weather gets cooler. This process is triggered by the decrease in daylight hours, typically around September or October in the Northern Hemisphere.\n\nHere's what happens:\n\n1. **Shorter days**: As the Earth continues its orbit around the Sun, the amount of sunlight that reaches the Earth's surface decreases. In the fall, the days get shorter, which means there are fewer hours of daylight for plants to photosynthesize.\n2. **Cooler temperatures**: As the weather cools down, the trees slow down their growth rate and prepare for winter. This is because they need to conserve energy and resources to survive the cold temperatures.\n3. **Increased sugar concentration**: In the fall, trees stop producing as much chlorophyll (the green pigment that helps plants absorb sunlight

In [3]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model=model)

In [4]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """\
<|start_header_id|>system<|end_header_id|>
You are a helpful assistant. You answer user questions based on context. If you can't answer the question with the context, say you don't know.
Context:
{context}
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
User Question:
{query}
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [5]:
rag_chain = rag_prompt | llm

In [6]:
rag_chain.invoke({"query" : "How old is Carl?", "context" : "Carl is a sweet dude, he's 40."})

'Carl is 40 years old.'

In [7]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [8]:
import os

files = os.listdir("data")
files = [os.path.join("data", file) for file in files]

print(files[0])

data/Planning for Care Costs _ Alzheimer's Association.html


In [9]:
from langchain_community.document_loaders import BSHTMLLoader

docs = []
for file in files:
    loader = BSHTMLLoader(file) 
    docs.append(loader.load())

print(docs[0])

[Document(metadata={'source': "data/Planning for Care Costs _ Alzheimer's Association.html", 'title': "\n\tPlanning for Care Costs | Alzheimer's Association\n"}, page_content='\n\n\tPlanning for Care Costs | Alzheimer\'s Association\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCall our 24 hours, seven days a week helpline at 800.272.3900\n\n\n\n\n\nmenu\n\n\nAbout\nNews\nEvents\nProfessionals\nEn Español\nE-News\n\n\n\n\n\n24/7 Helpline\n\n800.272.3900\n\nDonate\n\n\n\n\n\n\nAlzheimer\'s & Dementia\n\n\n\nWhat is Alzheimer’s Disease?Brain TourYounger/Early-Onset Alzheimer\'sMythsWhat Are the Causes and Risk Factors of Alzheimer\'s and Other Dementias?Is Alzheimer\'s Genetic?Women and Alzheimer\'sWhat Is Dementia?Types of DementiaCreutzfeldt-Jakob DiseaseDementia with Lewy BodiesDown Synd

In [41]:
from langchain_qdrant import QdrantVectorStore

url = "192.168.2.42:6334"
qdrant = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=True,
    collection_name="Caregiver_Docs",
)

In [42]:
qdrant_vectorstore = QdrantVectorStore.from_existing_collection(
    embeddings=embeddings,
    collection_name="DnD_Documents",
    url=url
)

TypeError: QdrantVectorStore.__init__() got an unexpected keyword argument 'url'